In [1]:
import numpy as np

from dipy.core.gradients import gradient_table
from dipy.data import get_fnames, default_sphere
from dipy.io.gradients import read_bvals_bvecs
from dipy.io.image import load_nifti
from dipy.denoise.patch2self import patch2self

hardi_fname, hardi_bval_fname, hardi_bvec_fname = get_fnames('stanford_hardi')

data1, affine = load_nifti(hardi_fname)

# NOTE:
# if comparison needs to be done with P2S2, uncomment the following

# data1, affine = load_nifti('cs_denoised_SH_50K.nii.gz') 

bvals, bvecs = read_bvals_bvecs(hardi_bval_fname, hardi_bvec_fname)

data = patch2self(data1, bvals, model='ols', shift_intensity=True,
                  clip_negative_vals=False, verbose=True)

gtab = gradient_table(bvals, bvecs)

Denoised b0 Volume:  0
Denoised b0 Volume:  1
Denoised b0 Volume:  2
Denoised b0 Volume:  3
Denoised b0 Volume:  4
Denoised b0 Volume:  5
Denoised b0 Volume:  6
Denoised b0 Volume:  7
Denoised b0 Volume:  8
Denoised b0 Volume:  9
Denoised DWI Volume:  0
Denoised DWI Volume:  1
Denoised DWI Volume:  2
Denoised DWI Volume:  3
Denoised DWI Volume:  4
Denoised DWI Volume:  5
Denoised DWI Volume:  6
Denoised DWI Volume:  7
Denoised DWI Volume:  8
Denoised DWI Volume:  9
Denoised DWI Volume:  10
Denoised DWI Volume:  11
Denoised DWI Volume:  12
Denoised DWI Volume:  13
Denoised DWI Volume:  14
Denoised DWI Volume:  15
Denoised DWI Volume:  16
Denoised DWI Volume:  17
Denoised DWI Volume:  18
Denoised DWI Volume:  19
Denoised DWI Volume:  20
Denoised DWI Volume:  21
Denoised DWI Volume:  22
Denoised DWI Volume:  23
Denoised DWI Volume:  24
Denoised DWI Volume:  25
Denoised DWI Volume:  26
Denoised DWI Volume:  27
Denoised DWI Volume:  28
Denoised DWI Volume:  29
Denoised DWI Volume:  30
Denoi

In [2]:
from dipy.reconst.csdeconv import (auto_response_ssst,
                                   mask_for_response_ssst,
                                   response_from_mask_ssst)

response, ratio = auto_response_ssst(gtab, data, roi_radii=10, fa_thr=0.7)

In [3]:
mask = mask_for_response_ssst(gtab, data, roi_radii=10, fa_thr=0.7)
nvoxels = np.sum(mask)
print(nvoxels)

response, ratio = response_from_mask_ssst(gtab, data, mask)

555


In [4]:
from dipy.viz import window, actor
from dipy.sims.voxel import single_tensor_odf

# Enables/disables interactive visualization
interactive = True

scene = window.Scene()
evals = response[0]
evecs = np.array([[0, 1, 0], [0, 0, 1], [1, 0, 0]]).T


response_odf = single_tensor_odf(default_sphere.vertices, evals, evecs)
# transform our data from 1D to 4D
response_odf = response_odf[None, None, None, :]
response_actor = actor.odf_slicer(response_odf, sphere=default_sphere,
                                  colormap='plasma')
scene.add(response_actor)
print('Saving illustration as csd_response.png')
window.record(scene, out_path='csd_response.png', size=(200, 200))
# if interactive:
#     window.show(scene)

Saving illustration as csd_response.png


In [5]:
scene.rm(response_actor)

In [6]:
from dipy.reconst.csdeconv import recursive_response
import dipy.reconst.dti as dti
tenmodel = dti.TensorModel(gtab)
tenfit = tenmodel.fit(data, mask=data[..., 0] > 200)

from dipy.reconst.dti import fractional_anisotropy
FA = fractional_anisotropy(tenfit.evals)
MD = dti.mean_diffusivity(tenfit.evals)
wm_mask = (np.logical_or(FA >= 0.4, (np.logical_and(FA >= 0.15, MD >= 0.0011))))

response = recursive_response(gtab, data, mask=wm_mask, sh_order=8,
                              peak_thr=0.01, init_fa=0.08,
                              init_trace=0.0021, iter=8, convergence=0.001,
                              parallel=False)

In [7]:
interactive = True
response_signal = response.on_sphere(default_sphere)
# transform our data from 1D to 4D
response_signal = response_signal[None, None, None, :]
response_actor = actor.odf_slicer(response_signal, sphere=default_sphere,
                                  colormap='plasma')

scene = window.Scene()

scene.add(response_actor)
print('Saving illustration as csd_recursive_response.png')
window.record(scene, out_path='csd_recursive_response.png', size=(500, 500))
# if interactive:
#     window.show(scene)

Saving illustration as csd_recursive_response.png


In [8]:
scene.rm(response_actor)

In [9]:
from dipy.reconst.csdeconv import ConstrainedSphericalDeconvModel
csd_model = ConstrainedSphericalDeconvModel(gtab, response)
data_small = data[20:50, 55:85, 38:39]
csd_fit = csd_model.fit(data_small)
csd_odf = csd_fit.odf(default_sphere)

fodf_spheres = actor.odf_slicer(csd_odf, sphere=default_sphere, scale=0.9,
                                norm=False, colormap='plasma')

scene.add(fodf_spheres)

print('Saving illustration as csd_odfs.png')
window.record(scene, out_path='csd_odfs_den_sketch.png', size=(600, 600))
if interactive:
    window.show(scene)

Saving illustration as csd_odfs.png
